# Calibration of stereo camera
- Basically recycling Week 4 code
- link to get images: https://campuscvut-my.sharepoint.com/:f:/g/personal/maleklu6_cvut_cz/EnyU2LTN6QFOlGlts1-sSusB6t9Q6hAcSeDAQd2se_cJlQ?e=gff67m


## Parameters
* change parameters here to adjust the path and part of codes that are executed

In [39]:
import cv2
import numpy as np
import glob
from matplotlib import pyplot as plt
import time

In [40]:
rerun_calibration = False           # Already calibrated, could remain False
rerun_undistortion = False          # Undistorts images defined in path
rerun_rectification = True          # Rectify images in path
rerun_rect_uncalib = False        # Rectify images in path
rerun_depth = False                  # Create disparity map from rectified images
path_calib = 'images/calibration/original/'
path_dist_in = 'images/sample_without_occlusion/original/'
path_dist_out = 'images/sample_without_occlusion/undistorted/'
path_rect_in = 'images/sample_without_occlusion/original/' #undistorted images should be used for uncalibrated rectification
path_rect_out = 'images/sample_without_occlusion/rectified/'
path_rect_uncalib_out = 'images/sample_without_occlusion/rectified_uncalibrated/'
path_depth_in = 'images/sample_without_occlusion/rectified/'
path_depth_out = 'images/sample_without_occlusion/depth/'


## CALIBRATION
* Values from previous runs. Used if rerun_calib is set to false (needed in the next part)
* No need to run the calibratino again

In [41]:
K_left = np.array([[705.12855386,   0,         621.0422932 ],
 [  0 ,        705.05638672, 370.57066306],
 [  0  ,         0      ,     1  ,      ]])
K_right = np.array([[702.64805575  , 0     ,    649.52345869],
 [  0       ,  702.90821064, 373.12894423],
 [  0      ,     0       ,    1        ]])
dist_left = np.array([[-3.29479779e-01 , 1.41779399e-01 ,-1.15869227e-04 , 2.53564192e-04
  -3.10092442e-02]])
dist_right = np.array([[-3.25580130e-01 , 1.39151531e-01 ,-2.55232895e-04 , 4.20204047e-04
  -3.19659396e-02]])
R = np.array([[ 0.99991381, -0.00530365, -0.01201018],
 [ 0.00527804 , 0.99998373, -0.00216356],
 [ 0.01202145 , 0.00209999 , 0.99992553]])
T = np.array([[-1.19993826e+02],
 [-2.56957545e-01],
 [-5.18613288e-02]])
F = np.array([[-6.02253356e-09 ,1.09798538e-07 ,-4.24691775e-04],
 [ 2.97380893e-06 , 5.39499206e-07 , 1.78876466e-01],
 [-1.67147433e-03 ,-1.80656239e-01 , 1.00000000e+00]])

### Function

In [42]:
def calibration(debug):

    # Write the image names
    images_left = glob.glob(path_calib+'left-*.png')
    images_right = glob.glob(path_calib+'right-*.png')
    assert images_left
    assert images_right

    # Implement the number of vertical and horizontal corners
    nb_vertical = 9
    nb_horizontal = 6
    checker_size = 33.6 # size of square in mm

    # Prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((nb_horizontal*nb_vertical, 3), np.float32)
    objp[:, :2] = np.mgrid[0:nb_vertical, 0:nb_horizontal].T.reshape(-1, 2)
    objp *= checker_size #results in transformation T in mm units in real world

    # Arrays to store object points and image points from all the images.
    objpoints = []  # 3d point in real world space
    imgpoints_left = []  # 2d points in image plane.
    imgpoints_right = []  # 2d points in image plane.

    for i in range(0,len(images_left)):

        # Load the image
        gray_left = cv2.cvtColor(cv2.imread(images_left[i]), cv2.COLOR_BGR2GRAY)
        gray_right = cv2.cvtColor(cv2.imread(images_right[i]), cv2.COLOR_BGR2GRAY)

        # Implement findChessboardCorners here
        ret_left, corners_left = cv2.findChessboardCorners(gray_left, (nb_vertical, nb_horizontal))
        ret_right, corners_right = cv2.findChessboardCorners(gray_right, (nb_vertical, nb_horizontal))

        # If found, add object points, image points (after refining them)
        if ret_left == True and ret_right==True:
            objpoints.append(objp)
            imgpoints_left.append(corners_left)
            imgpoints_right.append(corners_right)

    # Get the camera matrix
    _, K_left, dist_left, _, _ = cv2.calibrateCamera(objpoints, imgpoints_left, gray_left.shape[::-1], None, None)
    _, K_right, dist_right, _, _ = cv2.calibrateCamera(objpoints, imgpoints_right, gray_right.shape[::-1], None, None)

    # Get the excentric parameters between stereo lenses
    '''Camera matrix output and distortion is the same as input'''
    _, _, _, _, _, R, T, _, F = cv2.stereoCalibrate(objpoints, imgpoints_left, imgpoints_right, K_left, dist_left, K_right, dist_right, gray_left.shape[::-1], None, None, None, None, cv2.CALIB_RATIONAL_MODEL)

    # Print calibrated values
    if debug:
        print('K_left',K_left)
        print('K_right',K_right)
        print('dist_left',dist_left)
        print('dist_right',dist_right)
        print('R',R)
        print('T',T)
        print('F', F)
    return K_left, K_right, dist_left, dist_right, R, T, F

### Program

In [43]:
if rerun_calibration:
    K_left, K_right, dist_left, dist_right, R, T, F = calibration(debug=True)


## UNDISTORTION

### Function

In [44]:
def undistort(side,K,dist):
    if side == 'left':
        path_in = path_dist_in+'left/*.png'
        path_out = path_dist_out+'left/left-'
    elif side == 'right':
        path_in = path_dist_in+'right/*.png'
        path_out = path_dist_out+'right/right-'

    images = glob.glob(path_in)
    assert images

    # Get optimal intrinsic matrix based on parameter alpha (used for rectification?)
    '''if alpha>0, the undistorted result is likely to have some black pixels corresponding to "virtual" pixels outside of the captured distorted image'''
    (h,w) = cv2.imread(images[0]).shape[0:2]
    K_new, roi = cv2.getOptimalNewCameraMatrix(K, dist,(w,h), alpha=0)
    
    i = 0
    for fname in images:
        # Undistort
        img = cv2.imread(fname)
        dst = cv2.undistort(img, K, dist, None, K_new)

        # Crop the image
        x, y, w, h = roi
        dst = dst[y:y+h, x:x+w]

        # Save image
        cv2.imwrite(path_out+str(i)+'.png', dst)
        i += 1

    print("Undistortion "+side+ " done")

### Program

In [45]:
if rerun_undistortion:
    undistort('left',K_left,dist_left)
    undistort('right',K_right,dist_right)

## RECTIFICATION

### Functions


In [46]:
def draw_lines(img1, img2, lines, pts1, pts2):
    ''' img1 - image on which we draw the epilines for the points in img2
        lines - corresponding epilines '''
    (r, c) = img1.shape
    img1 = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR)
    img2 = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)
    for r, pt1, pt2 in zip(lines, pts1, pts2):
        color = tuple(np.random.randint(0, 255, 3).tolist())
        x0, y0 = map(int, [0, -r[2]/r[1]])
        x1, y1 = map(int, [c, -(r[2]+r[0]*c)/r[1]])
        img1 = cv2.line(img1, (x0, y0), (x1, y1), color, 2)
        img1 = cv2.circle(img1, tuple(pt1), 5, color, -1)
        img2 = cv2.circle(img2, tuple(pt2), 5, color, -1)
    return img1, img2

def draw_epipolar_lines(img_left, img_right):
    '''Draws epipolar lines to the image'''
    
    # Change to RGB
    gray_left = cv2.cvtColor(img_left, cv2.COLOR_BGR2GRAY)
    gray_right = cv2.cvtColor(img_right, cv2.COLOR_BGR2GRAY)

    # Find the keypoints and descriptors with SIFT
    sift = cv2.SIFT_create()
    kp_left, des_left = sift.detectAndCompute(gray_left, None)
    kp_right, des_right = sift.detectAndCompute(gray_right, None)

    # Match points
    matches = cv2.BFMatcher().match(des_left, des_right)
    matches = sorted(matches, key=lambda x: x.distance)
    nb_matches = 200  # Using 200 best matches
    good = []
    pts1 = []
    pts2 = []
    for m in matches[:nb_matches]:
        good.append(m)
        pts1.append(kp_left[m.queryIdx].pt)
        pts2.append(kp_right[m.trainIdx].pt)
    pts1 = np.int32(pts1)
    pts2 = np.int32(pts2)

    # Get fundamental matrix
    F, inliers = cv2.findFundamentalMat(pts1, pts2, method=cv2.FM_RANSAC)

    # Remove outliers
    pts1 = pts1[inliers.ravel() == 1]
    pts2 = pts2[inliers.ravel() == 1]

    # Draw lines
    lines1 = cv2.computeCorrespondEpilines(pts2.reshape(-1, 1, 2), 2, F)
    lines1 = lines1.reshape(-1, 3)
    epilines_left, keypoints_left = draw_lines(gray_left, gray_right, lines1, pts1, pts2)
    lines2 = cv2.computeCorrespondEpilines(pts1.reshape(-1, 1, 2), 1, F)
    lines2 = lines2.reshape(-1, 3)
    epilines_right, keypoints_right = draw_lines(gray_right, gray_left, lines2, pts2, pts1)

    fig, axs = plt.subplots(1, 2, constrained_layout=True, figsize=(10, 10))
    axs[0].imshow(epilines_left)
    axs[0].set_title('left epipolar lines')
    axs[1].imshow(epilines_right)
    axs[1].set_title('right epipolar lines')
    plt.show()

def rectify(img_left, img_right, K_left, dist_left, K_right, dist_right, R, T, leftMapX, leftMapY,rightMapX, rightMapY, debug=False):
    '''Rectify the image'''

    left_rectified = np.zeros(img_left.shape[:2], np.uint8)
    right_rectified = np.zeros(img_right.shape[:2], np.uint8)
    left_rectified = cv2.remap(img_left, leftMapX, leftMapY, cv2.INTER_LINEAR, left_rectified, cv2.BORDER_CONSTANT)
    right_rectified = cv2.remap(img_right, rightMapX, rightMapY, cv2.INTER_LINEAR, right_rectified, cv2.BORDER_CONSTANT)

    if debug:
        draw_epipolar_lines(left_rectified,right_rectified)

    return left_rectified, right_rectified

def rectify_uncalibrated(img_left, img_right, F, debug=False):  
    ''' We use the fundamental matrix from calibration. Since the camera is not moving, F should be the same as the one from calibration. If we compute F here, we of course get new F values, as we "calibrate" it on different iamges'''

    # Find the keypoints and descriptors with SIFT
    sift = cv2.SIFT_create()
    kp_left, des_left = sift.detectAndCompute(img_left, None)
    kp_right, des_right = sift.detectAndCompute(img_right, None)

    # Match points
    matches = cv2.BFMatcher().match(des_left, des_right)
    matches = sorted(matches, key=lambda x: x.distance)
    nb_matches = 200  # Using 200 best matches
    good = []
    pts1 = []
    pts2 = []
    for m in matches[:nb_matches]:
        good.append(m)
        pts1.append(kp_left[m.queryIdx].pt)
        pts2.append(kp_right[m.trainIdx].pt)
    pts1 = np.int32(pts1)
    pts2 = np.int32(pts2)
    
    # Rectify images
    (h,w,_) = img_left.shape
    _, H1, H2 = cv2.stereoRectifyUncalibrated(np.float32(pts1), np.float32(pts2), F, (w,h))
    left_rectified = cv2.warpPerspective(img_left, H1, (w,h))
    right_rectified = cv2.warpPerspective(img_right, H2, (w,h))

    if debug:
        draw_epipolar_lines(left_rectified,right_rectified)

    return left_rectified,right_rectified

### Rectification

In [47]:
if rerun_rectification:

    # Read the undistorted images
    imagesL = glob.glob(path_rect_in+'left/*.png')
    imagesR = glob.glob(path_rect_in+'right/*.png')
    assert imagesL
    assert imagesR

    # Get the mapping matrix (nearly the same for all images, the differences are not that signifficant)
    (h,w,_) = cv2.imread(imagesL[0]).shape
    R_left, R_right, P_left, P_right, _, roi_left, roi_right = cv2.stereoRectify(K_left, dist_left, K_right, dist_right, (w,h), R,T, alpha=0) #0.22 # CALIB_ZERO_DISPARITY
    #TODO: Unused mising paramter Q: disparity to depth matching matrix = after getting disparit map, we can call reprojectImageTo3D() and get the 3D model     
    #TODO: Use last two parameter (roi_left, roi_right) to crop the image

    #  Find the Maping Matrices
    leftMapX, leftMapY = cv2.initUndistortRectifyMap(K_left, dist_left, R_left, P_left, (w,h), cv2.CV_32FC1)
    rightMapX, rightMapY = cv2.initUndistortRectifyMap(K_right, dist_right, R_right, P_right, (w,h), cv2.CV_32FC1)

    # print(leftMapX,leftMapY)
    # print(rightMapX,rightMapY)

    for i in range(0, len(imagesL)):
        left_rectified, right_rectified = rectify(cv2.imread(imagesL[i]), cv2.imread(imagesR[i]), K_left, dist_left, K_right, dist_right, R, T, leftMapX, leftMapY,rightMapX, rightMapY, debug=False) 

        # Save images into folder
        cv2.imwrite(path_rect_out+'left/left-'+str(i)+'.png', left_rectified)
        cv2.imwrite(path_rect_out+'right/right-'+str(i)+'.png', right_rectified)
    print("Calibrated Rectification done")

if rerun_rect_uncalib:

    # Read the undistorted images
    imagesL = glob.glob(path_rect_in+'left/*.png')
    imagesR = glob.glob(path_rect_in+'right/*.png')
    assert imagesL
    assert imagesR

    for i in range(0, len(imagesL)):
        left_rectified, right_rectified = rectify_uncalibrated(cv2.imread(imagesL[i]), cv2.imread(imagesR[i]), F, debug=False) 

        # Save images into folder
        cv2.imwrite(path_rect_uncalib_out+'left/left-'+str(i)+'.png', left_rectified)
        cv2.imwrite(path_rect_uncalib_out+'right/right-'+str(i)+'.png', right_rectified)
    print("Uncalibrated Rectification done")


## IMAGE DEPTH

### Function


In [ ]:
def depth_map(debug):

    # Read the undistorted images
    imagesL = glob.glob(path_depth_in+'left/*.png')
    imagesR = glob.glob(path_depth_in+'right/*.png')
    assert imagesL
    assert imagesR

    # Iterate through the images
    for i in range(0, len(imagesL)):
        img_left = cv2.imread(path_depth_in+'left/left-'+str(i)+'.png')
        img_right = cv2.imread(path_depth_in+'right/right-'+str(i)+'.png')
        gray_left = cv2.cvtColor(img_left, cv2.COLOR_BGR2GRAY)
        gray_right = cv2.cvtColor(img_right, cv2.COLOR_BGR2GRAY)

        # Parameters
        min_disp = 0  # 22
        num_disp = 16*14  # 256
        block_size = 5  #5 
        sigma = 7 #1.5
        lmbda = 16000.0 #8000

        # Disparity Map
        stereo_left = cv2.StereoBM_create(numDisparities=num_disp, blockSize=block_size)
        stereo_right = cv2.ximgproc.createRightMatcher(stereo_left)
        stereo_left.setMinDisparity(min_disp)
        stereo_left.setDisp12MaxDiff(1)  # 200
        stereo_left.setUniquenessRatio(1)  # 1
        stereo_left.setSpeckleRange(1)  # 10
        stereo_left.setSpeckleWindowSize(1)  # 3
        disp_left = stereo_left.compute(gray_left, gray_right)#.astype(np.float32)
        disp_left2 = cv2.normalize(disp_left, None, 255, 0, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
        disp_right = stereo_right.compute(gray_right,gray_left)

        # WLSFilter
        wls_filter = cv2.ximgproc.createDisparityWLSFilter(stereo_left)
        wls_filter.setLambda(lmbda)
        wls_filter.setSigmaColor(sigma)
        disp_filtered = wls_filter.filter(disp_left, gray_left, disparity_map_right=disp_right)
        disp_filtered[disp_filtered<-16] = -16
        disp_filtered = (disp_filtered+16)/8

        # Compute the Z coordinate
        '''We look at the area around the centroid in depth image to get the Z coordinate'''
        x = 1250
        y = 360
        Z = np.mean(disp_filtered[y-15:y+15,x-15:x+15]) # h,w format
        
        # Save image
        cv2.imwrite(path_depth_out+'depth_filtered'+str(i)+'.png', disp_filtered)

        if debug:
            print(np.min(disp_filtered),np.max(disp_filtered),np.mean(disp_filtered))
            f, (ax1, ax2, ax3, ax4, ax5, ax6) = plt.subplots(1, 6, figsize=(30, 30))
            ax1.imshow(gray_left,cmap='gray')
            ax2.imshow(gray_right,cmap='gray')
            ax3.imshow(disp_left,cmap='gray')
            ax4.imshow(disp_left2,cmap='gray')
            ax5.imshow(disp_right,cmap='gray')
            ax6.imshow(disp_filtered,cmap='gray')
            plt.show()


### Program


In [ ]:
if rerun_depth:
    depth_map(debug=False)